In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
from tqdm import tqdm
import json
import random
import torch
import numpy as np
from transformers import AutoTokenizer, AutoConfig, DebertaV2ForSequenceClassification

In [3]:
config = AutoConfig.from_pretrained('mesolitica/malaysian-debertav3-base')

In [4]:
config.problem_type = "single_label_classification"
config.label2id = {'contradiction': 0, 'entailment': 1}

In [5]:
model = DebertaV2ForSequenceClassification.from_pretrained('mesolitica/malaysian-debertav3-base', config = config)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at mesolitica/malaysian-debertav3-base and are newly initialized: ['pooler.dense.weight', 'classifier.bias', 'pooler.dense.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
_ = model.cuda()

In [7]:
tokenizer = AutoTokenizer.from_pretrained('mesolitica/malaysian-debertav3-base')

In [8]:
trainable_parameters = [param for param in model.parameters() if param.requires_grad]
trainer = torch.optim.AdamW(trainable_parameters, lr = 2e-5)

In [9]:
train_X, train_Y = [], []
with open('shuffled-train.json') as fopen:
    for l in fopen:
        l = json.loads(l)
        train_X.append(l['src'])
        train_Y.append(l['label'])

In [10]:
test_X, test_Y = [], []
with open('shuffled-test.json') as fopen:
    for l in fopen:
        l = json.loads(l)
        test_X.append(l['src'])
        test_Y.append(l['label'])
        
len(test_X)

16037

In [11]:
batch_size = 8
epoch = 100

In [ ]:
best_dev_acc = -np.inf
patient = 1
current_patient = 0

for e in range(epoch):
    pbar = tqdm(range(0, len(train_X), batch_size))
    losses = []
    for i in pbar:
        trainer.zero_grad()
        x = train_X[i: i + batch_size]
        y = np.array(train_Y[i: i + batch_size])
        
        padded = tokenizer(x, truncation = True, padding = True, return_tensors = 'pt', max_length = 1024)
        padded['labels'] = torch.from_numpy(y)
        for k in padded.keys():
            padded[k] = padded[k].cuda()
        
        padded.pop('token_type_ids', None)
            
        loss, pred = model(**padded, return_dict = False)
        loss.backward()
        
        # grad_norm = torch.nn.utils.clip_grad_norm_(trainable_parameters, 1.0)
        trainer.step()
        losses.append(float(loss))
        pbar.set_postfix(loss = float(loss))
        
        
    dev_predicted = []
    for i in range(0, len(test_X[:10000]), batch_size):
        x = test_X[i: i + batch_size]
        y = np.array(test_Y[i: i + batch_size])
        padded = tokenizer(x, truncation = True, padding = True, return_tensors = 'pt', max_length = 1024)
        padded['labels'] = torch.from_numpy(y)
        for k in padded.keys():
            padded[k] = padded[k].cuda()
            
        padded.pop('token_type_ids', None)
        
        loss, pred = model(**padded, return_dict = False)
        dev_predicted.append((pred.argmax(axis = 1).detach().cpu().numpy() == y).mean())
        
    dev_predicted = np.mean(dev_predicted)
    
    print(f'epoch: {e}, loss: {np.mean(losses)}, dev_predicted: {dev_predicted}')
    
    if dev_predicted >= best_dev_acc:
        best_dev_acc = dev_predicted
        current_patient = 0
        model.save_pretrained('small')
    else:
        current_patient += 1
    
    if current_patient >= patient:
        break

  2%|▎                      | 2160/136696 [03:50<3:23:19, 11.03it/s, loss=0.688]

In [15]:
real_Y = []
for i in tqdm(range(0, len(test_X), batch_size)):
    x = test_X[i: i + batch_size]
    y = np.array(test_Y[i: i + batch_size])
    padded = tokenizer(x, padding = 'longest', return_tensors = 'pt')
    padded['labels'] = torch.from_numpy(y)
    for k in padded.keys():
        padded[k] = padded[k].cuda()

    loss, pred = model(**padded, return_dict = False)
    real_Y.extend(pred.argmax(axis = 1).detach().cpu().numpy().tolist())

100%|██████████████████████████████████████| 2005/2005 [00:19<00:00, 100.76it/s]


In [16]:
from sklearn import metrics

print(
    metrics.classification_report(
        real_Y, test_Y,
        digits = 5
    )
)

              precision    recall  f1-score   support

           0    0.79792   0.78841   0.79314      7803
           1    0.80173   0.81078   0.80623      8234

    accuracy                        0.79990     16037
   macro avg    0.79983   0.79960   0.79969     16037
weighted avg    0.79988   0.79990   0.79986     16037



In [17]:
tokenizer.model_input_names = ['input_ids', 'attention_mask']

In [18]:
tokenizer.push_to_hub('mesolitica/finetune-mnli-debertav3-small', safe_serialization = True)

CommitInfo(commit_url='https://huggingface.co/mesolitica/finetune-mnli-debertav3-small/commit/bee1a18c8fcb630d040c8932ca0d1e05fc91f9cc', commit_message='Upload tokenizer', commit_description='', oid='bee1a18c8fcb630d040c8932ca0d1e05fc91f9cc', pr_url=None, pr_revision=None, pr_num=None)

In [19]:
model.push_to_hub('mesolitica/finetune-mnli-debertav3-small', safe_serialization = True)

model.safetensors:   0%|          | 0.00/169M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mesolitica/finetune-mnli-debertav3-small/commit/ed780b39fbfb632504d70c55e6da12dc895d7b83', commit_message='Upload DebertaV2ForSequenceClassification', commit_description='', oid='ed780b39fbfb632504d70c55e6da12dc895d7b83', pr_url=None, pr_revision=None, pr_num=None)